#### Algoritmo Naive Bayes
Criar um moldelo para prever se o cliente será um bom pagador

In [36]:
# Importando as Bilbliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score

#### Carregamento da base de dados e definição dos previsores
(variáveis independentes -x) e a classe (variável dependente -y)

In [37]:
# Mapear o caminho do nome do arquivo
path = 'C:\\Users\\FIC\\Documents\\Luiz Antonio Python\\Machine Learning-20231004T233458Z-001\\Machine Learning\\'
file = 'Credit.csv'

# Atribuindo o arquivo a um dataframe
credito = pd.read_csv(path + file)

credito.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


In [38]:
credito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   checking_status         1000 non-null   object
 1   duration                1000 non-null   int64 
 2   credit_history          1000 non-null   object
 3   purpose                 1000 non-null   object
 4   credit_amount           1000 non-null   int64 
 5   savings_status          1000 non-null   object
 6   employment              1000 non-null   object
 7   installment_commitment  1000 non-null   int64 
 8   personal_status         1000 non-null   object
 9   other_parties           1000 non-null   object
 10  residence_since         1000 non-null   int64 
 11  property_magnitude      1000 non-null   object
 12  age                     1000 non-null   int64 
 13  other_payment_plans     1000 non-null   object
 14  housing                 1000 non-null   object
 15  exist

In [39]:
credito.shape

(1000, 21)

In [40]:
# Sparação de dados
previsores = credito.iloc[:,0:20].values
classe = credito.iloc[:,20].values

In [41]:
credito.iloc[:,0:20]

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,4,'real estate',67,none,own,2,skilled,1,yes,yes
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,2,'real estate',22,none,own,1,skilled,1,none,yes
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,3,'real estate',49,none,own,1,'unskilled resident',2,none,yes
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,4,'life insurance',45,none,'for free',1,skilled,2,none,yes
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,4,'no known property',53,none,'for free',2,skilled,2,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,'no checking',12,'existing paid',furniture/equipment,1736,<100,4<=X<7,3,'female div/dep/mar',none,4,'real estate',31,none,own,1,'unskilled resident',1,none,yes
996,<0,30,'existing paid','used car',3857,<100,1<=X<4,4,'male div/sep',none,4,'life insurance',40,none,own,1,'high qualif/self emp/mgmt',1,yes,yes
997,'no checking',12,'existing paid',radio/tv,804,<100,>=7,4,'male single',none,4,car,38,none,own,1,skilled,1,none,yes
998,<0,45,'existing paid',radio/tv,1845,<100,1<=X<4,4,'male single',none,4,'no known property',23,none,'for free',1,skilled,1,yes,yes


### Tranformar os atributos categóricos em atributos númericos

Dvemos passar o índice da coluna categórica , e criar um objeto para cada coluna
(atributo). Na sequência será executado o processo de encoding para regitrar o teste

In [42]:
# Primeira coluna
labelencoder1 = LabelEncoder()
previsores[:,0] = labelencoder1.fit_transform(previsores[:,0])

#Coluna 2
labelencoder2 = LabelEncoder()
previsores[:,2] = labelencoder1.fit_transform(previsores[:,2])
previsores[:,2]


array([1, 3, 1, 3, 2, 3, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 1, 4, 3, 3, 1, 3,
       1, 1, 1, 3, 4, 0, 3, 2, 3, 3, 3, 1, 3, 1, 1, 3, 3, 3, 3, 3, 2, 1,
       1, 1, 3, 3, 1, 3, 2, 2, 3, 3, 2, 0, 3, 1, 3, 1, 3, 1, 4, 4, 3, 3,
       3, 3, 3, 3, 3, 1, 1, 1, 3, 1, 3, 2, 4, 3, 3, 3, 3, 3, 3, 1, 1, 3,
       3, 4, 1, 1, 1, 1, 3, 4, 1, 3, 1, 2, 3, 3, 2, 1, 3, 1, 0, 3, 3, 3,
       2, 3, 3, 1, 3, 1, 3, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3, 3, 1, 1, 3, 2,
       3, 3, 3, 1, 2, 3, 1, 3, 3, 3, 3, 3, 1, 0, 1, 1, 1, 1, 3, 3, 3, 1,
       2, 3, 1, 0, 3, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 1, 4, 3, 2, 3, 2, 0,
       3, 1, 3, 1, 2, 2, 0, 1, 1, 1, 0, 1, 3, 4, 3, 4, 3, 3, 3, 1, 1, 3,
       3, 2, 1, 3, 2, 3, 1, 1, 1, 1, 3, 3, 1, 3, 4, 2, 1, 1, 4, 3, 3, 3,
       3, 3, 3, 2, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3,
       4, 1, 0, 3, 1, 4, 3, 3, 1, 3, 3, 2, 3, 2, 1, 0, 3, 1, 3, 3, 1, 1,
       1, 3, 1, 3, 3, 3, 3, 1, 0, 3, 3, 3, 1, 3, 3, 3, 1, 3, 0, 1, 1, 3,
       3, 2, 3, 2, 3, 3, 1, 1, 1, 3, 3, 3, 3, 1, 3,

In [43]:
# primeira coluna
labelencoder1 = LabelEncoder()
previsores[:,0] = labelencoder1.fit_transform(previsores[:,0])

# Coluna 02
labelencoder2 = LabelEncoder()
previsores[:,2] = labelencoder1.fit_transform(previsores[:,2])

# Coluna 03
labelencoder3 = LabelEncoder()
previsores[:, 3] = labelencoder3.fit_transform(previsores[:, 3])

# Coluna 05
labelencoder4 = LabelEncoder()
previsores[:, 5] = labelencoder4.fit_transform(previsores[:, 5])

# Coluna 06
labelencoder5 = LabelEncoder()
previsores[:, 6] = labelencoder5.fit_transform(previsores[:, 6])

# Coluna 08
labelencoder6 = LabelEncoder()
previsores[:, 8] = labelencoder6.fit_transform(previsores[:, 8])

# Coluna 09
labelencoder7 = LabelEncoder()
previsores[:, 9] = labelencoder7.fit_transform(previsores[:, 9])

# Coluna 11
labelencoder8 = LabelEncoder()
previsores[:, 11] = labelencoder8.fit_transform(previsores[:, 11])

#Coluna 13
labelencoder9 = LabelEncoder()
previsores[:, 13] = labelencoder9.fit_transform(previsores[:, 13])

# Coluna 14
labelencoder10 = LabelEncoder()
previsores[:, 14] = labelencoder10.fit_transform(previsores[:, 14])

# Coluna 16
labelencoder11 = LabelEncoder()
previsores[:, 16] = labelencoder11.fit_transform(previsores[:, 16])

# Coluna 18
labelencoder12 = LabelEncoder()
previsores[:, 18] = labelencoder12.fit_transform(previsores[:, 18])

# Coluna 19
labelencoder13 = LabelEncoder()
previsores[:, 19] = labelencoder13.fit_transform(previsores[:, 19])

### Divisão da base de dados entre treinamentos e teste
(30% para) testar e 70% para treinar)


In [48]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,
                                                                  classe,
                                                                  test_size = 0.3,
                                                                  random_state = 0)

## Criação e treinamento do modelo
(geração de tabela de probabilidade)

In [49]:

naive_bayes = GaussianNB()
naive_bayes.fit(X_treinamento, y_treinamento)

GaussianNB()

### Previsão utilizando os registros de teste

In [52]:
previsoes = naive_bayes.predict(X_teste)
previsoes 

array(['bad', 'good', 'good', 'good', 'bad', 'good', 'good', 'good',
       'good', 'bad', 'bad', 'bad', 'good', 'bad', 'good', 'good', 'good',
       'good', 'bad', 'good', 'bad', 'good', 'bad', 'good', 'good', 'bad',
       'good', 'good', 'good', 'bad', 'good', 'good', 'good', 'good',
       'good', 'bad', 'good', 'good', 'good', 'good', 'good', 'bad',
       'good', 'good', 'good', 'bad', 'bad', 'bad', 'bad', 'bad', 'good',
       'bad', 'good', 'good', 'good', 'good', 'bad', 'good', 'good',
       'good', 'bad', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'bad', 'good', 'good', 'good', 'good', 'good', 'bad',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'bad', 'good', 'good', 'bad', 'bad', 'good', 'bad', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'bad', 'good', 'good', 'bad', 'bad',
       'good', 'good', 'good', 'bad', 'good', 'good', 'bad', 'good',
      

In [53]:
previsoes = naive_bayes.predict(X_teste)
X_teste

array([[2, 36, 3, ..., 1, 1, 1],
       [0, 9, 3, ..., 2, 0, 0],
       [0, 18, 3, ..., 1, 1, 1],
       ...,
       [0, 24, 1, ..., 1, 0, 1],
       [1, 27, 4, ..., 1, 1, 1],
       [1, 12, 3, ..., 1, 0, 1]], dtype=object)

## Testar com um arquivo novo crédito


In [56]:
novoCredito = pd.read_csv(path + 'NovoCredit.csv')
novoCredito.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,'no checking',12,'existing paid',radio/tv,804,<100,>=7,4,'male single',none,4,car,38,none,own,1,skilled,1,none,yes


In [61]:
novo_credito = novoCredito.iloc[:, 0:20].values

novo_credito[:,0] = labelencoder1.fit_transform(novo_credito[:,0])
novo_credito[:, 2] = labelencoder2.fit_transform(novo_credito[:, 2])
novo_credito[:, 3] = labelencoder3.fit_transform(novo_credito[:, 3])
novo_credito[:, 5] = labelencoder4.fit_transform(novo_credito[:, 5])
novo_credito[:, 6] = labelencoder5.fit_transform(novo_credito[:, 6])
novo_credito[:, 8] = labelencoder6.fit_transform(novo_credito[:, 8])
novo_credito[:, 9] = labelencoder7.fit_transform(novo_credito[:, 9])
novo_credito[:, 11] = labelencoder8.fit_transform(novo_credito[:, 11])
novo_credito[:, 13] = labelencoder9.fit_transform(novo_credito[:, 13])
novo_credito[:, 14] = labelencoder10.fit_transform(novo_credito[:, 14])
novo_credito[:, 16] = labelencoder11.fit_transform(novo_credito[:, 16])
novo_credito[:, 18] = labelencoder12.fit_transform(novo_credito[:, 18])
novo_credito[:, 19] = labelencoder13.fit_transform(novo_credito[:, 19])

In [60]:
#Resultado da previsão
naive_bayes.predict(novo_credito)

array(['good'], dtype='<U4')